In [7]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.constraints import *
from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.table.grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

import meshcat
import meshcat.geometry as meshcat_geom

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

Meshcat url:  http://127.0.0.1:7001/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [17]:
# Set up grammar and constraint set.
grammar = SpatialSceneGrammar(
    root_node_type = Table,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint()
]

def sample_realistic_scene(grammar, constraints, seed=None, skip_physics_constraints=False):
    if seed is not None:
        torch.random.manual_seed(seed)
    topology_constraints, continuous_constraints = split_constraints(constraints)
    if len(topology_constraints) > 0:
        tree, success = rejection_sample_under_constraints(grammar, topology_constraints, 1000)
        if not success:
            logging.error("Couldn't rejection sample a feasible tree config.")
            return None, None
    else:
        tree = grammar.sample_tree(detach=True)

    samples = do_fixed_structure_hmc_with_constraint_penalties(
        grammar, tree, num_samples=25, subsample_step=1,
        with_nonpenetration=False, zmq_url=vis.window.zmq_url,
        constraints=continuous_constraints,
        kernel_type="NUTS", max_tree_depth=6, target_accept_prob=0.8, adapt_step_size=True,
        structure_vis_kwargs={"with_triad": False, "linewidth": 30, "node_sphere_size": 0.02,
                             "alpha": 0.5}
    )

    # Step through samples backwards in HMC process and pick out a tree that satisfies
    # the constraints.
    good_tree = None
    best_bad_tree = None
    best_violation = None
    for candidate_tree in samples[::-1]:
        total_violation = eval_total_constraint_set_violation(candidate_tree, constraints)
        if total_violation <= 0.:
            good_tree = candidate_tree
            break
        else:
            if best_bad_tree is None or total_violation <= best_violation:
                best_bad_tree = candidate_tree
                best_violation = total_violation.detach()
            
    if good_tree == None:
        logging.error("No tree in samples satisfied constraints.")
        print("Best total violation: %f" % best_violation)
        print("Violations of best bad tree:")
        for constraint in constraints:
            print("constraint ", constraint, ": ", constraint.eval(best_bad_tree))
        return None, None

    if skip_physics_constraints:
        return None, good_tree

    feasible_tree = project_tree_to_feasibility(deepcopy(good_tree), do_forward_sim=True, timestep=0.001, T=1.)
    return feasible_tree, good_tree

In [18]:
test_tree, _ = sample_realistic_scene(grammar, constraints, seed=45)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")

Initial trace log prob:  tensor(-8082.3273)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Warmup:   3%|▎         | 1/37 [02:15, 135.43s/it, step size=1.53e-02, acc. prob=0.667]


In [5]:
# Save grammar state dict
state_dict_file = "target_dataset_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "target_dataset_examples.pickle"
N = 48
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    tree, _ = sample_realistic_scene(grammar, constraints)
    if tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump(tree, f)
        k += 1
        pbar.update(k)

Saving state dict to  target_dataset_grammar_state_dict.torch


Samples:   0%|          | 0/48 [00:00<?, ?it/s]

Initial trace log prob:  tensor(81.3814)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


...

Sample: 100%|██████████| 37/37 [01:05,  1.77s/it, step size=8.80e-04, acc. prob=0.923]


Initial trace log prob:  tensor(-1732.6501)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [00:44,  1.19s/it, step size=1.08e-03, acc. prob=0.874]


In [6]:
print(tree.score())

tensor([-126.6013])


# Build sample population from various stages of em fitting process

In [4]:
fit_grammar = SpatialSceneGrammar(
    root_node_type = Table,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.0, 0., 0.8]))
)
constraints = [
    ObjectsOnTableConstraint(),
    ObjectSpacingConstraint()
]
#fit_grammar.load_state_dict(torch.load("fit_grammar.torch"))
with open("fit_em.pickle", "rb") as fp:
    em = pickle.load(fp)
fit_grammar.load_state_dict(em.grammar_iters[-1])

<All keys matched successfully>

In [5]:
test_tree, _ = sample_realistic_scene(fit_grammar, constraints, seed=43)
if test_tree is not None:
    draw_scene_tree_contents_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/contents")
    draw_scene_tree_structure_meshcat(test_tree, zmq_url=vis.window.zmq_url,  prefix="test_tree/structure")

[2022-01-18 15:18:32.742] [console] [warning] FindResource ignoring DRAKE_RESOURCE_ROOT='/home/gizatt/drake' because it does not contain a 'drake' subdirectory.


Initial trace log prob:  tensor(-3.9916)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:04,  1.75s/it, step size=6.50e-04, acc. prob=0.879]


In [31]:
# This one saves out the pre-fit examples, and doesn't bother with physics,
# since these samples are all over the place. It still does HMC to try to resolve
# the basic constraints (for better comparison with post-fit)
fit_grammar.load_state_dict(em.grammar_iters[0])
SKIP_PHYSICS_CONSTRAINTS = True
state_dict_file = "pre_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "pre_fit_grammar_draws.pickle"
N = 50
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    print(k, N)
    _, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints)
    if nonfeasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((None, nonfeasible_tree), f)
        k += 1
        pbar.update(k)
pbar.close()

Saving state dict to  pre_fit_grammar_state_dict.torch


Samples:   0%|          | 0/50 [00:00<?, ?it/s]

Initial trace log prob:  tensor(-15099950.3582)
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [00:18,  1.96it/s, step size=1.46e-03, acc. prob=0.980]


Initial trace log prob:  tensor(-9358447.4980)


...

Initial trace log prob:  tensor(-19306929.5716)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Warmup:  32%|███▏      | 12/37 [00:05,  1.29it/s, step size=1.37e-03, acc. prob=0.762]

KeyboardInterrupt: 

In [6]:
# This one saves out the post-fit examples, before and after the physics projection step.
SKIP_PHYSICS_CONSTRAINTS = True
fit_grammar.load_state_dict(em.grammar_iters[-1])
state_dict_file = "post_fit_grammar_state_dict.torch"
print("Saving state dict to ", state_dict_file)
torch.save(fit_grammar.state_dict(), state_dict_file)
# Try to collect a target number of examples, and save them out
dataset_save_file = "post_fit_grammar_draws.pickle"
N = 50
k = 0
pbar = tqdm(total=N, desc="Samples")
while k < N:
    feasible_tree, nonfeasible_tree = sample_realistic_scene(fit_grammar, constraints)
    if feasible_tree is not None:
        with open(dataset_save_file, "a+b") as f:
            pickle.dump((feasible_tree, nonfeasible_tree), f)
        k = k + 1
        pbar.update(k)
pbar.close()

Saving state dict to  post_fit_grammar_state_dict.torch


Samples:   0%|          | 0/50 [00:00<?, ?it/s]

Initial trace log prob:  tensor(6.6754)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


...

Sample: 100%|██████████| 37/37 [00:35,  1.05it/s, step size=5.84e-04, acc. prob=0.951]


Initial trace log prob:  tensor(-5594.9709)


Warmup:   0%|          | 0/37 [00:00, ?it/s]

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


Sample: 100%|██████████| 37/37 [01:02,  1.68s/it, step size=6.74e-04, acc. prob=0.880]
